In [1]:
from functools import lru_cache
from pathlib import Path
from typing import List, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

import amazon_dataset
import movielens_dataset
import bookcrossing_dataset

# Intialize Datasets

In [2]:
def get_base_path(dataset: str) -> Path:
    if dataset.startswith('ml-'):
        return movielens_dataset.BASE_DATA_FOLDER
    elif dataset == 'bookcrossing':
        return  bookcrossing_dataset.BASE_DATA_FOLDER
    return amazon_dataset.BASE_DATA_FOLDER

In [3]:
#DATASET = 'Musical_Instruments'
#DATASET = 'Clothing_Shoes_and_Jewelry'
#DATASET = 'Home_and_Kitchen'
#DATASET = 'Movies_and_TV'
DATASET = 'ml-1m'
#DATASET = 'bookcrossing'

VALIDATION_SIZE=0.15
RANDOM_SEED = 20230219
MIN_RATING = 0

base_path = get_base_path(DATASET)

#ALEXNET_IMAGE_FEATURES = BASE_PATH / f'{DATASET}_alexnet_features.npz'
VIT_IMAGE_FEATURES = base_path / f'{DATASET}_vit_features.npz'
CLIP_IMAGE_FEATURES = base_path / f'{DATASET}_clipimage_features.npz'

CLIP_TEXT_FEATURES = base_path / f'{DATASET}_cliptext_features.npz'
BERT_TEXT_FEATURES = base_path / f'{DATASET}_bert_features.npz'


#assert ALEXNET_IMAGE_FEATURES.exists()
assert VIT_IMAGE_FEATURES.exists(), f"{VIT_IMAGE_FEATURES} does not exist"
assert CLIP_IMAGE_FEATURES.exists(), f"{CLIP_IMAGE_FEATURES} does not exist"
assert BERT_TEXT_FEATURES.exists(), f"{BERT_TEXT_FEATURES} does not exist"
assert CLIP_TEXT_FEATURES.exists(), f"{CLIP_TEXT_FEATURES} does not exist"

In [4]:
print(f"VIT features {VIT_IMAGE_FEATURES}")
print(f"CLIP Image features {CLIP_IMAGE_FEATURES}")
print(f"BERT Text features {BERT_TEXT_FEATURES}")
print(f"CLIP Text features {CLIP_TEXT_FEATURES}")

VIT features data/movielens/ml-1m_vit_features.npz
CLIP Image features data/movielens/ml-1m_clipimage_features.npz
BERT Text features data/movielens/ml-1m_bert_features.npz
CLIP Text features data/movielens/ml-1m_cliptext_features.npz


In [5]:
def generate_unique_ids(series: pd.Series) -> pd.Series:
    rng = np.random.default_rng(RANDOM_SEED)
    unique_ids = series.unique()
    return pd.Series(
        index=rng.permutation(unique_ids), 
        data=range(len(unique_ids))
    )

def df_stats(df: pd.DataFrame) -> str:
    n_items = len(df['item_id'].unique())
    n_users = len(df['user_id'].unique())
    sparsity = 1. * len(df) / (n_users * n_items)
    return f'{n_users} users {n_items} items ratings: {len(df)}. Sparsity {sparsity * 100:.3f}%'

def save_numerized(
    reviews: pd.DataFrame, 
    uids: pd.Series, 
    product_ids: pd.Series,
    user_id_column: str,
    item_ids_column: str,
    columns: List[str],
    dest: Path
):
    """Save a Dataframe following userids and product_ids"""
    joined = reviews.join(uids.to_frame(user_id_column), on='user_id')
    assert joined[user_id_column].isna().sum() == 0
    
    joined = joined.join(product_ids.to_frame(item_ids_column), on='item_id')
    assert joined[item_ids_column].isna().sum() == 0
    
    res = joined[columns]
    
    res.to_csv(dest, index=False)

    return res

def split_train_test_proportion(df: pd.DataFrame, test_prop=0.2):
    """
    Split the dataframe by reviewer and take exactly `test_prop` records
    for test and leave the rest for training
    """
    rng = np.random.default_rng(RANDOM_SEED)

    res = df.copy()
    res['rnd'] = rng.random(size=len(df))
    res['rnd_rank'] = res.groupby('user_id')['rnd'].rank(pct=True)
    condition = res['rnd_rank'] <= test_prop
    
    (_, train), (_, test) = res.groupby(condition)
    
    assert(len(set(train.index) & set(test.index)) == 0)

    return df.loc[train.index], df.loc[test.index]

In [13]:
def get_reviews_df(dataset: str, min_rating: int) -> pd.DataFrame:
    if dataset.startswith('ml-'):
        reviews = movielens_dataset.reviews_df(dataset)
    elif dataset == 'bookcrossing':
        reviews = bookcrossing_dataset.reviews_df()
    else:
        reviews = amazon_dataset.reviews_df(dataset)

    with_min_ratings = reviews[reviews['rating'] >= min_rating]
    # valid user ids
    user_id_size = with_min_ratings.groupby('user_id').size()
    valid_user_ids = user_id_size.loc[user_id_size >= 5].index
    return with_min_ratings[with_min_ratings['user_id'].isin(valid_user_ids)]

reviews_df = get_reviews_df(DATASET, min_rating=0)
reviews_df.sample(n=3)

,user_id,item_id,rating,timestamp
759557,4511,2959,3,2000-07-30 17:38:01
127663,824,1674,3,2000-11-28 02:15:46
216609,1315,858,5,2000-11-21 04:09:06


# Generate Data for SEM-MacridVAE & DMRL

Some data stats

In [14]:
print(DATASET)
print(df_stats(reviews_df))

ml-1m
6040 users 3706 items ratings: 1000209. Sparsity 4.468%


Sort some users randomly

In [15]:
user_ids = generate_unique_ids(reviews_df['user_id'])
user_ids

3853       0
1976       1
4814       2
1517       3
4650       4
        ... 
1294    6035
644     6036
1670    6037
2738    6038
102     6039
Length: 6040, dtype: int64

Same with products

In [16]:
item_ids = generate_unique_ids(reviews_df['item_id'])
item_ids

2845       0
2661       1
432        2
3420       3
453        4
        ... 
3142    3701
3742    3702
1044    3703
1945    3704
530     3705
Length: 3706, dtype: int64

Split train test and validation by user

In [17]:
train_reviews, validation_test_reviews = split_train_test_proportion(reviews_df, test_prop=0.4)
validation_reviews, test_reviews = split_train_test_proportion(validation_test_reviews, test_prop=0.5)

In [18]:
len(train_reviews)

602537

len(validation_reviews)

In [19]:
len(test_reviews)

197656

Sanity checks. All users are in the train validation and test sets!

In [20]:
set(train_reviews['user_id']) == set(test_reviews['user_id']) ==  set(validation_reviews['user_id'])

True

Check that the train set and the validation set all have at least one item (same check as above)

In [21]:
pd.concat([
    train_reviews.groupby('user_id').size().to_frame('train_size'),
    validation_reviews.groupby('user_id').size().to_frame('val_size'),
    test_reviews.groupby('user_id').size().to_frame('test_size')
], axis=1).describe()

,train_size,val_size,test_size
count,6040.000000,6040.000000,6040.000000
mean,99.757781,33.115232,32.724503
std,115.651463,38.557939,38.540047
min,12.000000,4.000000,4.000000
25%,27.000000,9.000000,8.000000
50%,58.000000,19.000000,19.000000
75%,125.000000,42.000000,41.000000
max,1389.000000,463.000000,462.000000


In [23]:
combinations = pd.DataFrame.from_records([
    ('vit', 'bert', VIT_IMAGE_FEATURES, BERT_TEXT_FEATURES),
    ('none', 'bert', None, BERT_TEXT_FEATURES),
    ('vit', 'none', VIT_IMAGE_FEATURES, None),
    ('clip', 'clip', CLIP_IMAGE_FEATURES, CLIP_TEXT_FEATURES),
    ('none', 'clip', None, CLIP_TEXT_FEATURES),
    ('clip', 'none', CLIP_IMAGE_FEATURES, None),
    ('none', 'none', None, None),
], columns=['image', 'text', 'image_features', 'text_features'])
combinations['dest_folder'] = combinations.apply(
    lambda x: Path(f'data/{DATASET}-{x["image"]}_{x["text"]}'), axis=1
)
combinations

,image,text,image_features,text_features,dest_folder
0,vit,bert,data/movielens/ml-1m_vit_features.npz,data/movielens/ml-1m_bert_features.npz,data/ml-1m-vit_bert
1,none,bert,None,data/movielens/ml-1m_bert_features.npz,data/ml-1m-none_bert
2,vit,none,data/movielens/ml-1m_vit_features.npz,None,data/ml-1m-vit_none
3,clip,clip,data/movielens/ml-1m_clipimage_features.npz,data/movielens/ml-1m_cliptext_features.npz,data/ml-1m-clip_clip
4,none,clip,None,data/movielens/ml-1m_cliptext_features.npz,data/ml-1m-none_clip
5,clip,none,data/movielens/ml-1m_clipimage_features.npz,None,data/ml-1m-clip_none
6,none,none,None,None,data/ml-1m-none_none


Generate users

In [24]:
for c in combinations.itertuples():
    c.dest_folder.mkdir(exist_ok=True)

    user_ids.to_csv(c.dest_folder / 'users.txt')
    item_ids.to_csv(c.dest_folder / 'items.txt', header=['item_id'])

In [25]:
NUMERIZED_COMMON_PARAMS = {
    'uids': user_ids, 
    'product_ids': item_ids,
    'user_id_column': 'user',
    'item_ids_column': 'item',
    'columns': ['user', 'item']
}

for c in combinations.itertuples():
    c.dest_folder.mkdir(exist_ok=True)

    save_numerized(
        train_reviews, 
        dest=c.dest_folder / 'train.txt',
        **NUMERIZED_COMMON_PARAMS
    )


In [26]:
for c in combinations.itertuples():
    c.dest_folder.mkdir(exist_ok=True)

    save_numerized(
        validation_reviews, 
        dest=c.dest_folder / 'validation.txt',
        **NUMERIZED_COMMON_PARAMS
    )


'Saved validation data'

'Saved validation data'

In [27]:
for c in combinations.itertuples():
    save_numerized(
        test_reviews, 
        dest=c.dest_folder / 'test.txt',
        **NUMERIZED_COMMON_PARAMS
    )
'Saved test data'

'Saved test data'

In [28]:
@lru_cache()
def copy_features(features_file: Optional[Path]):
    if features_file is None:
        print('Returning Zeros')
        return np.zeros((len(item_ids), 256), dtype=np.float32)

    print(f'Opening file {features_file}')
    with np.load(features_file) as features:
        some_embedding = next(iter(features.values()))
        embedding_shape, = some_embedding.shape
        array_shape = (len(item_ids),  embedding_shape)
        print(f'Initializing array {array_shape}')
        res = np.full(array_shape,  fill_value=np.nan)

        for item_id, idx in tqdm(item_ids.items(), 
                total=len(item_ids), unit_scale=True, unit='items', 
                desc='Copying features'):
            value = features.get(str(item_id))
            if value is None:
                print(f'Item {item_id} not found in features file')
                res[idx, :] = 0    
            else:
                assert np.isnan(value).sum() == 0, "Feature has NaN Values"
                res[idx, :] = value

        return res

for c in combinations.itertuples():
    print(c)
    features_array = copy_features(features_file=c.image_features)
    embed_dest = c.dest_folder / 'embed_image.npy'
    np.save(embed_dest, features_array)

    print(f'{str(embed_dest)}: {embed_dest.stat().st_size // 2**20}MiB')

copy_features.cache_clear()

Pandas(Index=0, image='vit', text='bert', image_features=PosixPath('data/movielens/ml-1m_vit_features.npz'), text_features=PosixPath('data/movielens/ml-1m_bert_features.npz'), dest_folder=PosixPath('data/ml-1m-vit_bert'))
Opening file data/movielens/ml-1m_vit_features.npz
Initializing array (3706, 1024)


Copying features:   0%|          | 0.00/3.71k [00:00<?, ?items/s]

Copying features: 100%|██████████| 3.71k/3.71k [00:01<00:00, 2.29kitems/s]


data/ml-1m-vit_bert/embed_image.npy: 28MiB
Pandas(Index=1, image='none', text='bert', image_features=None, text_features=PosixPath('data/movielens/ml-1m_bert_features.npz'), dest_folder=PosixPath('data/ml-1m-none_bert'))
Returning Zeros
data/ml-1m-none_bert/embed_image.npy: 3MiB
Pandas(Index=2, image='vit', text='none', image_features=PosixPath('data/movielens/ml-1m_vit_features.npz'), text_features=None, dest_folder=PosixPath('data/ml-1m-vit_none'))
data/ml-1m-vit_none/embed_image.npy: 28MiB
Pandas(Index=3, image='clip', text='clip', image_features=PosixPath('data/movielens/ml-1m_clipimage_features.npz'), text_features=PosixPath('data/movielens/ml-1m_cliptext_features.npz'), dest_folder=PosixPath('data/ml-1m-clip_clip'))
Opening file data/movielens/ml-1m_clipimage_features.npz
Initializing array (3706, 768)


Copying features: 100%|██████████| 3.71k/3.71k [00:01<00:00, 2.50kitems/s]


data/ml-1m-clip_clip/embed_image.npy: 21MiB
Pandas(Index=4, image='none', text='clip', image_features=None, text_features=PosixPath('data/movielens/ml-1m_cliptext_features.npz'), dest_folder=PosixPath('data/ml-1m-none_clip'))
data/ml-1m-none_clip/embed_image.npy: 3MiB
Pandas(Index=5, image='clip', text='none', image_features=PosixPath('data/movielens/ml-1m_clipimage_features.npz'), text_features=None, dest_folder=PosixPath('data/ml-1m-clip_none'))
data/ml-1m-clip_none/embed_image.npy: 21MiB
Pandas(Index=6, image='none', text='none', image_features=None, text_features=None, dest_folder=PosixPath('data/ml-1m-none_none'))
data/ml-1m-none_none/embed_image.npy: 3MiB


In [30]:
for c in combinations.itertuples():
    print(c)
    features_array = copy_features(features_file=c.text_features)
    embed_dest = c.dest_folder / 'embed_text.npy'
    np.save(embed_dest, features_array)

    print(f'{str(embed_dest)}: {embed_dest.stat().st_size // 2**20}MiB')

copy_features.cache_clear()

Pandas(Index=0, image='vit', text='bert', image_features=PosixPath('data/movielens/ml-1m_vit_features.npz'), text_features=PosixPath('data/movielens/ml-1m_bert_features.npz'), dest_folder=PosixPath('data/ml-1m-vit_bert'))
Opening file data/movielens/ml-1m_bert_features.npz
Initializing array (3706, 1024)


Copying features: 100%|██████████| 3.71k/3.71k [00:01<00:00, 2.41kitems/s]


data/ml-1m-vit_bert/embed_text.npy: 28MiB
Pandas(Index=1, image='none', text='bert', image_features=None, text_features=PosixPath('data/movielens/ml-1m_bert_features.npz'), dest_folder=PosixPath('data/ml-1m-none_bert'))
data/ml-1m-none_bert/embed_text.npy: 28MiB
Pandas(Index=2, image='vit', text='none', image_features=PosixPath('data/movielens/ml-1m_vit_features.npz'), text_features=None, dest_folder=PosixPath('data/ml-1m-vit_none'))
Returning Zeros
data/ml-1m-vit_none/embed_text.npy: 3MiB
Pandas(Index=3, image='clip', text='clip', image_features=PosixPath('data/movielens/ml-1m_clipimage_features.npz'), text_features=PosixPath('data/movielens/ml-1m_cliptext_features.npz'), dest_folder=PosixPath('data/ml-1m-clip_clip'))
Opening file data/movielens/ml-1m_cliptext_features.npz
Initializing array (3706, 768)


Copying features: 100%|██████████| 3.71k/3.71k [00:01<00:00, 2.46kitems/s]


data/ml-1m-clip_clip/embed_text.npy: 21MiB
Pandas(Index=4, image='none', text='clip', image_features=None, text_features=PosixPath('data/movielens/ml-1m_cliptext_features.npz'), dest_folder=PosixPath('data/ml-1m-none_clip'))
data/ml-1m-none_clip/embed_text.npy: 21MiB
Pandas(Index=5, image='clip', text='none', image_features=PosixPath('data/movielens/ml-1m_clipimage_features.npz'), text_features=None, dest_folder=PosixPath('data/ml-1m-clip_none'))
data/ml-1m-clip_none/embed_text.npy: 3MiB
Pandas(Index=6, image='none', text='none', image_features=None, text_features=None, dest_folder=PosixPath('data/ml-1m-none_none'))
data/ml-1m-none_none/embed_text.npy: 3MiB
